# BUFN400 Self Study Problems: Yield to Maturity and Duration
## By James Zhang

In [1]:
import pandas as pd
import numpy as np
import scipy
import warnings
warnings.filterwarnings("ignore")
import matplotlib.pyplot as plt

# Consider the Below Example

In [2]:
# Define assumptions

dtyp = np.float64

coupon = 3.00 # annual
par_value = 100.00 # percent of par
maturity = 2 #years (integer!)
Nfreq = 2 # bond-equivalent yield (integer!), 2 payments per year
price = 99.98

yield_min = 0.0200
yield_max = 0.0400
yield_num = 11
yield_inc = (yield_max - yield_min) / (yield_num - 1)

# Set up data as numpy arrays:

# ytm = np.arange(start=yield_min, stop=yield_max + 0.0001, step=yield_inc, dtype=dtyp)
ytm = np.linspace(start=yield_min, stop=yield_max, num=yield_num, endpoint=True, dtype=dtyp)

cf_num = maturity * Nfreq + 1
cf = np.full(shape=(cf_num,), fill_value=coupon / Nfreq, dtype=dtyp)
cf[0] = -price
cf[-1] = cf[-1] + par_value

t = np.linspace(start=0.00, stop=dtyp(maturity), num=cf_num, dtype=dtyp)

df = (1.00 / (1.00 + ytm / Nfreq)).reshape(yield_num, 1)**(Nfreq * t.reshape(1, cf_num))

# Perform calculations using numpy:

npv = df @ cf # net-present-value operator is linear ("@" means "matrix-vector product"!)
pv = -cf[0] + npv 

print(f"{coupon=}\n{par_value=}\n{maturity=}\n{Nfreq=}\n{price=}\n")
print(f"{t=}\n{cf=}\n{ytm=}\n{df=}\n")

dframe = pd.DataFrame({'yield' : ytm, 'PV' : pv, 'NPV' : npv})
dframe['yield_pct'] = dframe['yield'] * 100.00
dframe

coupon=3.0
par_value=100.0
maturity=2
Nfreq=2
price=99.98

t=array([0. , 0.5, 1. , 1.5, 2. ])
cf=array([-99.98,   1.5 ,   1.5 ,   1.5 , 101.5 ])
ytm=array([0.02 , 0.022, 0.024, 0.026, 0.028, 0.03 , 0.032, 0.034, 0.036,
       0.038, 0.04 ])
df=array([[1.        , 0.99009901, 0.98029605, 0.97059015, 0.96098034],
       [1.        , 0.98911968, 0.97835775, 0.96771291, 0.95718388],
       [1.        , 0.98814229, 0.97642519, 0.96484703, 0.95340615],
       [1.        , 0.98716683, 0.97449835, 0.96199245, 0.94964704],
       [1.        , 0.98619329, 0.97257721, 0.95914913, 0.94590644],
       [1.        , 0.98522167, 0.97066175, 0.95631699, 0.94218423],
       [1.        , 0.98425197, 0.96875194, 0.953496  , 0.93848032],
       [1.        , 0.98328417, 0.96684776, 0.95068609, 0.93479459],
       [1.        , 0.98231827, 0.96494919, 0.94788722, 0.93112693],
       [1.        , 0.98135427, 0.9630562 , 0.94509931, 0.92747725],
       [1.        , 0.98039216, 0.96116878, 0.94232233, 0.92384543

,yield,PV,NPV,yield_pct
0,0.020,101.950983,1.970983,2.0
1,0.022,101.556950,1.576950,2.2
2,0.024,101.164846,1.184846,2.4
3,0.026,100.774661,0.794661,2.6
4,0.028,100.386383,0.406383,2.8
5,0.030,100.000000,0.020000,3.0
6,0.032,99.615502,-0.364498,3.2
7,0.034,99.232877,-0.747123,3.4
8,0.036,98.852116,-1.127884,3.6
9,0.038,98.473205,-1.506795,3.8


# Problem 1
Study both the finance content and the Python/numpy syntax for the previous cell carefully to make sure you know all of the commands and concepts backwards and forwards.

1. Why is it better to use function np.linspace than the seemingly almost equivalent function np.arange (which is commented out)? Relatedly, why is the scalar 0.001 added to yield_max in the example?

**Solution:** It is better to use `np.linspace` then `np.arange` because `np.linspace` can use a non-integer step. It even recommends to do consider this in the Numpy documentation. In the example that is commented out that uses `np.arange`, adding 0.001 accounts for the non-integer step-size and allows `yield_max` to be included in the numbers.

2. When the yield to maturity of the security is equal to its coupon, is the present value exactly 100.00, or is the value of 100.00 in the dataframe a "coincidence" which depends on something specific to this example?

No, this is not a coincidence but rather a fundamental principle in bond pricing. Recall that the yield to maturity is the IRR of the bond, and the coupon rate fixed rate in which the issuer will deal the bondholder. If these two rates are the same, the bond is correctly priced (has zero NPV), and there's no incentive for traders to invest or short this bond. Thus, the bond price of 100.00 stays at its par value. 

3. Can you develop a one-sentence theory for why bond-equivalent yield y2 is used for calculations involving bonds with semi-annual coupons? What intuitive financial tradeoffs occur for choice between using $r_{\infty}$
and using $r_2$ (or $r_{12}$)?

Continuous compounding does not really make sense here because it is a semi-annual coupon, and intuitively the issuer will have to pay less of it is compounded every half year as opposed to continuously.


# Problem 2

In [4]:
epsilon = np.array([10.0**(-n) for n in range(10, 20)])

df = pd.DataFrame({'epsilon' : epsilon, 
                  '-epsilon?' : 1.00 - (1.00 + epsilon),
                  'one?' : (1.00 - (1.00 + epsilon)) / epsilon, 
                 'exp(epsilon)-1 ~ epsilon?' : np.exp(epsilon) - 1.00,
                 'np.expm1(epsilon) = epsilon!' : np.expm1(epsilon)})
df

,epsilon,-epsilon?,one?,exp(epsilon)-1 ~ epsilon?,np.expm1(epsilon) = epsilon!
0,1.000000e-10,-1.000000e-10,-1.000000,1.000000e-10,1.000000e-10
1,1.000000e-11,-1.000000e-11,-1.000000,1.000000e-11,1.000000e-11
2,1.000000e-12,-1.000089e-12,-1.000089,1.000089e-12,1.000000e-12
3,1.000000e-13,-9.992007e-14,-0.999201,9.992007e-14,1.000000e-13
4,1.000000e-14,-9.992007e-15,-0.999201,9.992007e-15,1.000000e-14
5,1.000000e-15,-1.110223e-15,-1.110223,1.110223e-15,1.000000e-15
6,1.000000e-16,0.000000e+00,0.000000,0.000000e+00,1.000000e-16
7,1.000000e-17,0.000000e+00,0.000000,0.000000e+00,1.000000e-17
8,1.000000e-18,0.000000e+00,0.000000,0.000000e+00,1.000000e-18
9,1.000000e-19,0.000000e+00,0.000000,0.000000e+00,1.000000e-19


In the cell above:

1. How small is epsilon when dramatic loss of precision occurs?

$\epsilon$ is $1e-13$ when dramatic losses of precision occur.

2. How small is epsilon when the nonzero result becomes numerically equal to zero?

$1e-16$ is when the nonzero result becomes numerically equal to 0.

3. Does `np.expm1` provide an accurate answer?

Yes, `np.exp1` works very well and his very precise.

I chose not to do the rest of the problems, as the notes says that they are "optional". Thus, I will move onto the Gordon Growth Formula section.